In [1]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, GPT2Tokenizer, GPT2Model 

from tqdm import tqdm
import random
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

# import model and tokenizer
model_name = 'EleutherAI/gpt-neo-2.7B'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPTNeoForCausalLM.from_pretrained(model_name)


In [ ]:
def ask_gpt_neo(question, max_length=150):
    """
    Function to ask a question to GPT-Neo and get the response.
    
    Args:
    question (str): A string containing the question or prompt for GPT-Neo.
    max_length (int): The maximum length of the tokens for the model output.
    
    Returns:
    str: The generated answer from GPT-Neo.
    """
    # Encode the question to tensor of integers using the tokenizer
    inputs = tokenizer.encode(question, return_tensors="pt")

    # Generate a response from the model
    outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1)

    # Decode the output tokens to string
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer

# Example prompt
prompt = "Based on the applicant's information: credit score of 720, annual income of $50,000, and requested loan amount of $10,000, should the loan be approved or denied? Please provide reasons for your decision."

# Get the response from GPT-Neo
response = ask_gpt_neo(prompt)
print("Response from GPT-Neo:", response)

In [ ]:
# Simple function to process the decision based on the model's output
def get_loan_decision(description):
    inputs = tokenizer(description, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits[:, -1, :], dim=-1)
        decision = "Approved" if predictions[0, tokenizer.encode('yes')[0]] > predictions[0, tokenizer.encode('no')[0]] else "Denied"
    return decision

# Example description
description = "Black man, stable income, secure job, good reliability, good salary"

# Get the loan decision
decision = get_loan_decision(description)
print("Loan Decision:", decision)